<h1>TRATAMENTO DE TEXTO - TESTE 7</h1>

#### importação e criação padrão

In [ ]:
# Importando as Bibliotecas

import numpy as np
import pandas as pd
import json
import random
import string


from textblob import TextBlob
from wordcloud import WordCloud
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Importando o conjunto de dados

data = pd.read_json("req.json", encoding='utf-8')
data.head(2)

In [ ]:
# Removendo colunas desnecessárias

columns_to_drop = ['_id', '_rev']
data.drop(columns_to_drop, axis=1, inplace=True)
data.head()

In [ ]:
# Removendo Caracteres Não ASCII
def remove_non_ascii_1(data):
    return ''.join([i if ord(i) < 128 else ' ' for i in data])

In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#### daqui pra cima padrão

In [ ]:
corpus = []
for i in range(0, 128):
    review = re.sub('[^a-zA-Z]', ' ', str(data['req'][i]))
    
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorized = CountVectorizer(stop_words='english')

X = vectorized.fit_transform(corpus)
y = vectorized.transform(data)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorized = CountVectorizer(stop_words='english')

X = vectorized.fit_transform(corpus).toarray()
data = vectorized.transform(data)
y = data[:128]

#### minimizando

In [ ]:
from sklearn.decomposition import NMF
model = NMF(init="nndsvd",
            n_components=10,
            max_iter=200)

# Get W and H, the factors
W = model.fit_transform(a)
H = model.components_

print("W:", W.shape)
print("H:", H.shape)

In [ ]:
vectorizer = vectorized

terms = [""] * len(vectorizer.vocabulary_)
for term in vectorizer.vocabulary_.keys():
    terms[vectorizer.vocabulary_[term]] = term
    
# Have a look that some of the terms
terms[-5:]

In [ ]:
for topic_index in range(H.shape[0]):  # H.shape[0] is k
    top_indices = np.argsort(H[topic_index,:])[::-1][0:10]
    term_ranking = [terms[i] for i in top_indices]
    print("Topic {}: {}".format(topic_index, ", ".join(term_ranking)))

#### normal

In [ ]:
# Criando o modelo Bag of Words

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(raw).toarray()
y = data.loc[:, 1].values

In [ ]:
# Dividindo o conjunto de dados no conjunto de treinamento e no conjunto de teste

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
# Adaptando Naive Bayes ao conjunto de treinamento

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

In [ ]:
# Previsão dos resultados do teste de teste

y_pred = classifier.predict(X_test)

In [ ]:
# Fazendo a matriz de confusão

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
# Aplicando a Validação Cruzada k-Fold

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 2)
accuracies.mean()
accuracies.std()